In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

from matplotlib import pyplot as plt
import scipy
from scipy.ndimage import gaussian_filter


import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

import cv2
import os

import scipy.misc
#import skimage.filter

import parmap

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc

clrs = ['red','blue','cyan','green','yellow','pink','magenta','black','lightgreen','lightblue']


Autosaving every 180 seconds


In [2]:
# ROTATE IMAGE
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

# FIND ROTATION 
def find_rotation(locs,
                  frame):
    
    padding_size = 400
    frame_padded = np.zeros((1024+padding_size*2, 1280+padding_size*2, 3), 'uint8')
       
    frame_padded[padding_size:padding_size+1024, padding_size:padding_size+1280] = frame
    #frame = np.pad(frame, 2, pad_with, padder=np.mean(frame))
    
    frame = frame_padded.copy()
    locs+=padding_size
    
    width = 100
    padding = 200
    # FIND CENTRES
    
    centrex = int(np.nanmean(locs[:,1]))
    centrey = int(np.nanmean(locs[:,0]))

    # GRAB CENTRED PATCH
    frame_patch = frame[centrex-width-padding:centrex+width+padding, 
                        centrey-width-padding:centrey+width+padding][:,:,1]
    
    # EXCLUDE PATCHES THAT ARE NOT RIGHT SIZE DUE TO CLIPPING;
    # TODO: fix this so it is not required
    if frame_patch.shape[0]!= (width*2 + padding*2):
        return None
    
    if frame_patch.shape[1]!= (width*2 + padding*2):
        return None

    # FIND ROTATION USING PCA
    x = locs[:,0]
    y = locs[:,1]
    x = x - np.mean(x)
    y = y - np.mean(y)
    coords = np.vstack([x, y])
    cov = np.cov(coords)
    evals, evecs = np.linalg.eig(cov)

    sort_indices = np.argsort(evals)[::-1]
    x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
    x_v2, y_v2 = evecs[:, sort_indices[1]]
    theta = np.arctan((x_v1)/(y_v1))  

    # ROTATE RAW DATA
    try:
        frame_rotated = rotate_image(frame_patch, -theta*180/3.14159)
        frame_rotated = frame_rotated[padding:-padding, padding: -padding]
    except:
        print ("Couldn't rotate or clip image")
        #return frame_rotated
        pass
    
    return frame_rotated

# GENERATE TRAINING IMAGES
def generate_training_image_cc(tracesx,
                               tracesy,
                               frame,
                               root_dir, 
                               id_,
                               features,
                               chain_id,
                               plotting,
                               save,
                               chain_id_ctr
                               ):
    
    #print (id_, chain_id)
    chain_id.append([])
    chain_id_ctr.append([])
    for k in range(0, tracesx.shape[0], 14):
        
        # add a placeholder
        chain_id[id_].append(np.nan)
        chain_id_ctr[id_].append(np.nan)
        #print (chain_id)
            
        ##############################################################
        ###################### GET X,Y LOCS ##########################
        ##############################################################
        tempx = np.float32(tracesx[k:k+features,id_])
        idx = np.where(tempx==0)[0]
        tempy = np.float32(tracesy[k:k+features,id_])
        locs = np.vstack((tempx,tempy))
        locs = np.delete(locs, idx, 1).T
        #print ("LOCS: ", locs)
        # if no locs left exit
        
        if locs.shape[0]<2:
            chain_id[id_][k//14] = np.nan
            chain_id_ctr[id_][k//14] = np.nan
            continue
        
        ##############################################################
        ################### ROTATE IMG ###############################
        ##############################################################
        frame_rotated = find_rotation(locs,
                                      frame)
        
        if frame_rotated is None:
            chain_id[id_][k//14] = np.nan
            chain_id_ctr[id_][k//14] = np.nan
            continue

        ##############################################################
        ##################### LINK CHAINS ############################
        ##############################################################
        # FIND CHAIN ID for this network:
        if np.isnan(chain_id[id_-1][k//14]):
            # need to find new chain id value
            temp = np.hstack(chain_id)
            #print (temp)
            chain_id[id_][k//14] = int(np.nanmax(temp)+1)
            chain_id_ctr[id_][k//14] = 0
            
        else:
            chain_id[id_][k//14] = chain_id[id_-1][k//14]
            chain_id_ctr[id_][k//14] = chain_id[id_-1][k//14]+1
             
        if save:
            continue
            
        try:
            local_dir = root_dir+str(chain_id[id_][k//14])
            os.mkdir(local_dir)
            #print ("NEW LOCLA DIR: ", local_dir)
        except:
            pass
        
        out_fname = root_dir+str(chain_id[id_][k//14])+'/frame_'+str(id_).zfill(7)+"_id_"+str(k//14)+'.npy'

        if plotting:
            fig=plt.figure(figsize=(10,10))
            plt.imshow(frame_rotated)
            plt.xticks([])
            plt.yticks([])
            plt.savefig(out_fname[:-4]+'.png')
            plt.close()

        np.save(out_fname[:-4]+'.npy', frame_rotated)

        #ax1 = plt.subplot(2,3,k//14+1)
        
    return chain_id, chain_id_ctr
        #ax1.imshow(res)

     


In [10]:
# GENERATE TRAINING DATA;
matplotlib.use('Agg')

# set start at frame_
id_ = 0
features = 10  # Exclude tails
root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/'

# LOAD VIDEO
video_name =root_dir + '/2020-3-16_12_57_12_418305_compressed.avi'
original_vid = cv2.VideoCapture(video_name)

# LOAD CC OUTPUTED TRACES
fname_traces_cc = root_dir + '2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89989.npz'
traces_cc = np.load(fname_traces_cc)
tracesx = traces_cc['tracesx']
print ("tracesx: ", tracesx.shape)
tracesy = traces_cc['tracesy']


# set chain_id for 3D CC at start
chain_id = []
chain_id.append(np.int32(np.arange(tracesx.shape[0]/14))) # start with potentially all the networks from the beginning, 
chain_id_ctr = []
chain_id_ctr.append(np.zeros(tracesx.shape[0]//14,'int32'))
print (chain_id_ctr)

root_dir = root_dir+'/training_images/'   

for chain in chain_id[0]:
    out_fname = root_dir+str(chain)
    try:
        os.mkdir(out_fname)
    except:
        pass
    
from tqdm import tqdm

# FOR CC need to have history, so start with following frame

plotting = False
save = False


# START WITH SECOND FRAME OTHERWISE 
ids = np.arange(1,tracesx.shape[1],1)
frame_start = ids[0]
original_vid.set(cv2.CAP_PROP_POS_FRAMES, frame_start)
for id_ in tqdm(ids):
    
    ret, frame = original_vid.read()
    chain_id, chain_id_ctr = generate_training_image_cc(tracesx,
                               tracesy,
                               frame,
                               root_dir,
                               id_,
                               features,
                               chain_id,
                               plotting,
                               save,
                               chain_id_ctr)
    
# SAVE CHAIN ID INFORMATION
fname_chain_id = fname_traces_cc[:-4]+"_chain_id.npy"
np.save(fname_chain_id, 
       chain_id)


  0%|          | 10/89988 [00:00<16:06, 93.08it/s]

tracesx:  (70, 89989)
[array([0, 0, 0, 0, 0], dtype=int32)]


100%|██████████| 89988/89988 [27:28<00:00, 54.58it/s] 


In [11]:
# fname_traces_cc = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz'
# fname_chain_id = fname_traces_cc[:-4]+"_chain_id.npy"
# np.save(fname_chain_id, 
#        chain_id)

In [15]:

temp = np.vstack(chain_id)
print (temp[-10:])

[[1157.   nan 1029. 1169.  773.]
 [1157. 1261. 1029. 1169.  773.]
 [1157.   nan 1029. 1169.  773.]
 [1157. 1262. 1029. 1169.  773.]
 [1157. 1262. 1029. 1169.  773.]
 [1157.   nan 1029. 1169.  773.]
 [1157.   nan 1029. 1169.  773.]
 [1157.   nan 1029. 1169.  773.]
 [1157. 1263. 1029. 1169.  773.]
 [1157. 1263. 1029. 1169.  773.]]


In [ ]:
# OPTIONAL: RUN ABOVE WITH PLOTTING OFF 
# THE PARSE THE chain_id variable for long chains and only save those

In [ ]:
######################################################################
############## MAKE CLASSIFIER TRAINING DATA FROM SAVED ##############
######################################################################
import numpy as np

root_animals = root_dir + '/training_images_cc/segments_training/'
names = ['female','male','pup1','pup2', 'block']

import glob

data = []
for ctr, name in enumerate(names):
    data.append([])

    fnames = glob.glob(root_animals+name+'/**/*',recursive = True)
    for fname in fnames:
        if '.npy' in fname:
            data[ctr].append(np.load(fname))

for k in range(len(names)):
    print (names[k], len(data[k]))
    temp = np.array(data[k])
    print (temp.shape)
    idx_rand = np.random.choice(np.arange(temp.shape[0]), 
                                #595, 
                                temp.shape[0]//2,
                                replace=False)
    print (k, idx_rand.shape)
    temp = np.array(temp[idx_rand])
    print (temp.shape)
    np.save(root_animals+"id_"+str(k)+'_train.npy', temp)
    
    # save test datasets; 
    temp = np.array(data[k])
    idx = np.arange(temp.shape[0])
    idx_test = np.delete(idx, idx_rand)
    print ("Idx test: ", idx_test.shape)
    
    temp = temp[idx_test]
    print ("saved test: ", temp.shape)
    np.save(root_animals+"id_"+str(k)+'_test.npy', temp)
    
    

female 1042
(1042, 200, 200)
0 (521,)
(521, 200, 200)
Idx test:  (521,)
saved test:  (521, 200, 200)
male 595
(595, 200, 200)
1 (297,)
(297, 200, 200)
Idx test:  (298,)
saved test:  (298, 200, 200)
pup1 1653
(1653, 200, 200)
2 (826,)
(826, 200, 200)
Idx test:  (827,)
saved test:  (827, 200, 200)
pup2 2449
(2449, 200, 200)
3 (1224,)
(1224, 200, 200)
Idx test:  (1225,)
saved test:  (1225, 200, 200)


(100, 100, 50)


In [252]:
# VISUALIZE MATCHES
clrs = ['red','blue','cyan','green','yellow','pink','magenta','white','lightgreen','lightblue']


for k in frames:  #skip first match
    print ("frame: ", k)
    im = surfaces_watershed[k]
    idx0 = np.where(im==0)
    #print ("idx0: ", idx0)
    im[idx0]==np.nan
    
    # 
    ids = np.unique(im)
    #print (ids)
    for ctr, id_ in enumerate(ids):
        if id_==0:
            continue

        #
        idx = np.where(im==id_)
        centrex = np.mean(idx[0])
        centrey = np.mean(idx[1])
        print (centrex.shape, centrex)
        
        #centrey = np.mean(im[idx[:,1]])

        match_id = np.array(matches[k][ctr]).squeeze()
        print ("matchid: ", match_id)
        
        plt.scatter(k, centrex+centrey, 
                    c=clrs[match_id%10])
        
plt.show()
        
        

frame:  0
() 20.92491467576792
matchid:  1
() 39.075
matchid:  3
() 62.41414141414141
matchid:  4
() 84.26804123711341
matchid:  5
frame:  1
() 17.873303167420815
matchid:  1
() 30.59722222222222
matchid:  1
() 39.14646464646464
matchid:  3
() 62.38693467336683
matchid:  4
() 84.29473684210527
matchid:  5
frame:  2
() 22.23017902813299
matchid:  1
() 38.775401069518715
matchid:  3
() 62.51269035532995
matchid:  4
() 84.2258064516129
matchid:  5
frame:  3
() 22.31782945736434
matchid:  1
() 38.776595744680854
matchid:  2
() 62.51269035532995
matchid:  4
() 84.23684210526316
matchid:  5
frame:  4
() 22.40776699029126
matchid:  1
() 38.71584699453552
matchid:  2
() 62.759803921568626
matchid:  3
() 84.14778325123153
matchid:  4
frame:  5
() 22.395734597156398
matchid:  1
() 38.73157894736842
matchid:  2
() 62.67464114832536
matchid:  4
() 84.09661835748793
matchid:  5
frame:  6
() 22.27972027972028
matchid:  1
() 39.20873786407767
matchid:  2
() 62.748768472906406
matchid:  4
() 84.317948

TypeError: unsupported operand type(s) for %: 'NoneType' and 'int'

In [215]:
id_ = 145
print (np.unique(surfaces_watershed[id_-1]))
print (np.unique(surfaces_watershed[id_]))
print (matches[id_])


[0 4 6 7]
[0 3 5 6]
[[0], [4], [6], [7]]


In [94]:
##################################################################
#################### EXTRACT PATCHES AND ALIGN THEM ##############
##################################################################

#import skimage.filter


fig=plt.figure()


#img = misc.imread('oval.png', flatten=1)

video_name ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'


original_vid = cv2.VideoCapture(video_name)

frame_id= 900
original_vid.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
ret, frame = original_vid.read()

img1 = img_array1[frame_id]

aspect = 'equal'
n_rows = np.unique(img1).shape[0]
for ctr_id, id_ in enumerate(np.unique(img1)):
    
    img = img_array1[frame_id].copy()
    
    #id_ = 5
    print ("ctr_ID: ", ctr_id)

    # PLOT FRAME
    ax=plt.subplot(n_rows,5,1+ctr_id*5)
    plt.imshow(frame)
    print (frame.shape)
    
    # PLOT FRAME 
    ax=plt.subplot(n_rows,5,2+ctr_id*5)
    plt.imshow(img, aspect=aspect)

    
    # ZERO OUT ALL NON ID_ values for PCA ROTATION COMPUTATION
    print (np.unique(img1))
    idx1 = np.where(img1==id_)
    idx2 = np.where(img1!=id_)
    img[idx1]=1
    img[idx2]=0


    width = 125
    padding = 30
    print ("idx1: ", idx1[0],
                    idx1[1])
    centrex = int(np.nanmean(idx1[0]))*8
    centrey = int(np.nanmean(idx1[1]))*8
    print ("cenres:", centrex, centrey,
          centrex//8, centrey//8)

    #frame_patch = np.zeros((200,200),'int32')
    frame_patch = frame[centrex-width-padding:centrex+width+padding, centrey-width-padding:centrey+width+padding][:,:,1]

    ax=plt.subplot(n_rows,5,3+ctr_id*5)
    plt.imshow(frame_patch,aspect=aspect)
    
    
    #
    y, x = np.nonzero(img)
    x = x - np.mean(x)
    y = y - np.mean(y)
    coords = np.vstack([x, y])

    cov = np.cov(coords)
    evals, evecs = np.linalg.eig(cov)

    sort_indices = np.argsort(evals)[::-1]
    x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
    x_v2, y_v2 = evecs[:, sort_indices[1]]


    # PLOT THE PATCH WITH ANIMAL IN IT
    ax=plt.subplot(n_rows,5,4+ctr_id*5)
    scale = 20
    plt.plot([x_v1*-scale*2, x_v1*scale*2],
             [y_v1*-scale*2, y_v1*scale*2], color='red')
    plt.plot([x_v2*-scale, x_v2*scale],
             [y_v2*-scale, y_v2*scale], color='blue')
    plt.plot(x, y, 'k.')
    plt.axis('equal')
    plt.gca().invert_yaxis()  # Match the image system with origin at top left

    #fig=plt.figure()

    theta = np.arctan((x_v1)/(y_v1))  
    rotation_mat = np.matrix([[np.cos(theta), -np.sin(theta)],
                          [np.sin(theta), np.cos(theta)]])
    transformed_mat = rotation_mat * coords

    # plot the transformed blob
    x_transformed, y_transformed = transformed_mat.A
    plt.plot(x_transformed, y_transformed, 'g.')


    # ROTATE RAW DATA
    ax=plt.subplot(n_rows, 5,5+ctr_id*5)
    try: 
        frame_rotated = rotate_image(frame_patch, -theta*180/3.14159)
        frame_rotated = frame_rotated[padding:-padding, padding: -padding]
        plt.imshow(frame_rotated,aspect=aspect)
    except:
        pass
plt.show()


IndexError: list index out of range

In [38]:

# ROTATE IMAGE
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result


# FIND CENTRES
locs = np.array([[100,145], [115,145], [125,155], [130,160]])

# FIND ROTATION USING PCA
x = locs[:,0]
y = locs[:,1]
x = x - np.mean(x)
y = y - np.mean(y)
coords = np.vstack([x, y])
cov = np.cov(coords)
evals, evecs = np.linalg.eig(cov)

sort_indices = np.argsort(evals)[::-1]
x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
x_v2, y_v2 = evecs[:, sort_indices[1]]
theta = np.arctan((x_v1)/(y_v1))  

# CHECK THAT HEAD ALWAYS UP
test_img = np.zeros((200,200),'float32')#+np.nan
buffer =2
for k in range(locs.shape[0]):
    test_img[int(locs[k,0])-buffer:int(locs[k,0])+buffer,
             int(locs[k,1])-buffer:int(locs[k,1])+buffer] = k+1

test_img_rotated = rotate_image(test_img, -theta*180/3.14159)

out = []
for k in range(locs.shape[0]):
    idx = np.where(test_img_rotated==k+1)
    out.append(np.mean(idx[0]))
out=np.array(out)
diff = out[1:]-out[:1]
if np.mean(diff)>0:
    print ("head up")
else:
    
    
    

4.0 0.0
19.56026936026936
head up
